# [Density-functional toolkit (DFTK)](https://github.com/JuliaMolSim/DFTK.jl)

Project of
[Antoine Levitt](http://antoine.levitt.fr/) and [Michael F. Herbst](https://michael-herbst.com/) from Centre Inria de Paris and Ecole des Ponts ParisTech.

From its homepage.
*The density-functional toolkit, or short DFTK is a library of Julia routines for experimentation with plane-wave-based density-functional theory (DFT), as implemented in much larger production codes such as Abinit, Quantum Espresso and VASP. The main aim at the moment is to provide a platform to facilitate numerical analysis of algorithms and techniques related to DFT. For this we want to leverage as much of the existing developments in plane-wave DFT and the related ecosystems of Julia, Python or C codes as possible.*


It is using `scipy`, `spglib` (interface for programs in C), `matplotlib`, `pymatgen` from Python 3.

All examples form DFTK.jl site, you can just click on the link.

## [DFT calculations in DFTK: Silicon and Graphite](https://github.com/JuliaMolSim/DFTK.jl/blob/master/examples/Silicon_Graphite.ipynb)

## Setting up libraries

In [ ]:
# Activate source directory of DFTK
import Pkg

Pkg.activate("..")
Pkg.instantiate()

# Pkg.add("Plots")
using Plots

# Pkg.add("DFTK")
using DFTK

### Defining paramaters

In [ ]:
kgrid = [3, 3, 3]   # k-Point grid
Ecut = 15           # Kinetic energy cutoff in Hartree
temperature = 0.01  # Smearing temperature in Hartree (for metals)
T = Float64;        # Floating point type for computations

## Silicon, an Insulator

### Setting up lattices and structures

In [ ]:
a = 10.263141334305942  # Silicon lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.0]; [1 0 1.0]; [1 1 0.0]]

In [ ]:
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))

In [ ]:
?ElementPsp

In [ ]:
?Element

In [ ]:
typeof(Si)

In [ ]:
atoms = [Si => [ones(3)/8, -ones(3)/8]]

In [ ]:
n_bands = 6

model = model_LDA(Array{T}(lattice), atoms)

## Running an Self-Consistent Field (SCF) computation

### Discretise the model

In [ ]:
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid)

In [ ]:
?PlaneWaveBasis

### Run self-consistent field calculation

In [ ]:
scfres = self_consistent_field(basis, tol=1e-10)

In [ ]:
?self_consistent_field

In [ ]:
display(scfres.energies)

In [ ]:
println("\nkpt    occupation")
for ik in 1:length(basis.kpoints)
    println("  $ik    $(scfres.occupation[ik])")
end

## Post-process SCF results

### Compute bandstructure

In [ ]:
plot_bandstructure(scfres, n_bands)

### Plot of the density versus the norm of the reciprocal lattice points

In [ ]:
Gabs = vec([sum(abs, G) for G in G_vectors(basis)])
scatter(Gabs, vec(abs.(scfres.ρ.fourier)), yaxis=:log)

### Plot the density of state

In [ ]:
εs = collect(-0.3:0.005:0.5)

doses = DOS.(εs, Ref(basis), Ref(scfres.eigenvalues),T=temperature*4,
            smearing=DFTK.Smearing.MethfesselPaxton1())

In [ ]:
q = plot(εs, doses, label="DOS")

In [ ]:
vline!(q, [scfres.εF], label="Fermi level")


## Build graphite (a Metal)
## Note: This is not exactly the minimum-energy structure

In [ ]:
ÅtoBohr = 1.8897261246257702 # Convert Å to bohr
a = 1.228ÅtoBohr
b = 2.12695839ÅtoBohr
c = 7ÅtoBohr

lattice = [[a a 0]; [-b b 0]; [0 0 c]]
C = ElementPsp(:C, psp=load_psp("hgh/lda/C-q4"))

atoms = [C => [[0, 0, 1/4], [0, 0, 3/4],
                [1/3, 2/3, 1/4], [2/3, 1/3, 3/4]], ]

n_bands = 15

model = model_DFT(Array{T}(lattice), atoms, :lda_xc_teter93;
                  temperature=temperature, smearing=Smearing.MethfesselPaxton2())

## We repeat code used previously for Silicon insulator

In [ ]:
# Running an SCF computation
# Discretise the model

basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid)



# Run self-consistent field calculation
scfres = self_consistent_field(basis, tol=1e-10)

display(scfres.energies)
println("\nkpt    occupation")
for ik in 1:length(basis.kpoints)
    println("  $ik    $(scfres.occupation[ik])")
end



# Post-process SCF results
# Compute bandstructure
plot_bandstructure(scfres, n_bands)

In [ ]:
# Plot of the density versus the norm of the reciprocal lattice points
Gabs = vec([sum(abs, G) for G in G_vectors(basis)])
scatter(Gabs, vec(abs.(scfres.ρ.fourier)), yaxis=:log)

In [ ]:
# Plot the density of state
εs = collect(-0.3:0.005:0.5)
doses = DOS.(εs, Ref(basis), Ref(scfres.eigenvalues), T=temperature*4,
             smearing=DFTK.Smearing.MethfesselPaxton1())
q = plot(εs, doses, label="DOS")
vline!(q, [scfres.εF], label="Fermi level")

# [Graphene](https://github.com/JuliaMolSim/DFTK.jl/blob/master/examples/graphene.jl)

In [ ]:
kgrid = [4, 4, 1]
Tsmear = 0.0009500431544769484
Ecut = 15

lattice = [4.659533614391621 -2.3297668071958104 0.0;
           0.0 4.035274479829987 0.0;
           0.0 0.0 15.117809010356462]

C = ElementPsp(:C, psp=load_psp("hgh/pbe/c-q4"))
atoms = [C => [[0.0, 0.0, 0.0], [0.33333333333, 0.66666666667, 0.0]]]

model = model_DFT(lattice, atoms, [:gga_x_pbe, :gga_c_pbe];
                  temperature=Tsmear, smearing=Smearing.Gaussian())

In [ ]:
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid)

In [ ]:
# Run SCF
n_bands = 6
scfres = self_consistent_field(basis; n_bands=n_bands)

# Print obtained energies
println()
display(scfres.energies)

## [Gross-Pitaevskii 1D equation](https://github.com/JuliaMolSim/DFTK.jl/blob/master/examples/gross_pitaevskii.jl)

\begin{equation}
    -\frac{ 1 }{ 2 } \Delta \psi( x ) + V( x ) \psi( x )
    + \alpha | \psi( x ) | \psi( x )
    = \lambda \psi( x ),
    \quad
    || \psi ||_{ L^{ 2 } } = 1
\end{equation}

We emulate this with custom external potential $V( x )$, and a custom $xc$ term.

In [ ]:
using DFTK
using LinearAlgebra
using Plots

In [ ]:
Ecut = 4_000
const α = 2

### Nonlinearity: energy $C ∫ \rho^{ \alpha }$

In [ ]:
C = 1.0

# Unit cell. Having two lattice vectors as zero means a 1D system
a = 10
lattice = a .* [[1 0 0.0]; [0 0 0]; [0 0 0]]

### Potential $f( x ) = ( x - a/2 )^{ 2 }$

In [ ]:
f(x) = (x - a/2)^2

### Increse number of electrons for fun

In [ ]:
n_electrons = 1

### We add the needed terms

In [ ]:
terms = [Kinetic(),
         ExternalFromReal(X -> f(X[1])),
         PowerNonlinearity(C, α),
]

In [ ]:
model = Model(lattice; n_electrons=n_electrons,terms=terms,
              spin_polarisation= :spinless) # "spinless fermions"

In [ ]:
basis = PlaneWaveBasis(model, Ecut)

In [ ]:
scfres = direct_minimization(basis, x_tol=1e-8, f_tol=-1, g_tol=-1)

In [ ]:
println()
display(scfres.energies)

In [ ]:
x = a * range(0, 1, length=basis.fft_size[1]+1)[1:end-1]

###   Converged density

In [ ]:
ρ = real(scfres.ρ.real)[:, 1, 1]

### First kpoint, all $G$ components , first eigenvector

In [ ]:
ψ_fourier = scfres.ψ[1][:, 1]

In [ ]:
ψ = G_to_r(basis, basis.kpoints[1], ψ_fourier)[:, 1, 1]  # IFFT back to real space

In [ ]:
@assert sum(abs2.(ψ)) * (x[2] - x[1]) ≈ 1.0

### Phase fix

In [ ]:
ψ /= (ψ[div(end, 2)] / abs(ψ[div(end, 2)]))

$\psi( x )$ solves
$-\frac{ 1 }{ 2 } \Delta \psi( x ) + V_{ \textrm{ext} } \psi( x ) + C \alpha \rho^{ \alpha - 1 } \psi( x ) = \alpha \psi( x )$

In [ ]:
N = length(x)
A = Array(Tridiagonal(-ones(N - 1), 2ones(N), -ones(N - 1)))

In [ ]:
A[1, end] = A[end, 1] = -1
K = A / ((x[2] - x[1])^2) / 2

In [ ]:
V = Diagonal(f.(x) + C .* α .* (ρ.^(α - 1)))
H = K + V

In [ ]:
p = plot(x, real.(ψ), label="ψreal")
plot!(p, x, imag.(ψ), label="ψimag")
plot!(p, x, ρ, label="ρ")
plot!(p, x, abs.(H*ψ - dot(ψ, H*ψ)/dot(ψ, ψ)*ψ), label="residual")
gui(p)

# [Magnesium PBE](https://github.com/JuliaMolSim/DFTK.jl/blob/master/examples/magnesium_pbe.jl)

In [ ]:
using DFTK
using Plots

### Calculation parameters

In [ ]:
kgrid = [4, 4, 4]      # k-Point grid
Ecut = 15              # kinetic energy cutoff in Hartree
supercell = [1, 1, 1]  # Lattice supercell
n_bands = 8            # Number of bands for SCF and plotting
Tsmear = 0.01          # Smearing temperature in Hartree

### Setup magnesium lattice (constants in Bohr)

In [ ]:
a = 3.0179389193174084
b = 5.227223542397263
c = 9.773621942589742

lattice = [[-a -a 0]; [-b b 0]; [0 0 -c]]

In [ ]:
Mg = ElementPsp(:Mg, psp=load_psp("hgh/pbe/Mg-q2"))

In [ ]:
atoms = [Mg => [[2/3, 1/3, 1/4], [1/3, 2/3, 3/4]]]

### Make a supercell if desired

In [ ]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell(supercell)
lattice = load_lattice(pystruct)
atoms = [Mg => [s.frac_coords for s in pystruct.sites]]

### Setup PBE model with Methfessel-Paxton smearing and its discretisation

In [ ]:
model = model_DFT(lattice, atoms, [:gga_x_pbe, :gga_c_pbe];
                  temperature=Tsmear,
                  smearing=DFTK.Smearing.MethfesselPaxton1())

In [ ]:
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid)

### Run SCF

In [ ]:
scfres = self_consistent_field(basis, n_bands=n_bands)

### Print obtained energies and plot bands

In [ ]:
println()
display(scfres.energies)
p = plot_bandstructure(scfres, n_bands)

### Plot DOS

In [ ]:
εs = range(minimum(minimum(scfres.eigenvalues)) - 1,
                      maximum(maximum(scfres.eigenvalues)) + 1, length=1000)
Ds = DOS.(εs, Ref(basis), Ref(scfres.eigenvalues), T=Tsmear*4,
          smearing=DFTK.Smearing.MethfesselPaxton1())

In [ ]:
q = plot(εs, Ds, label="DOS")
    vline!(q, [scfres.εF], label="εF")

In [ ]:
gui(plot(p, q))